# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/ibtissammim/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 67 (delta 7), reused 5 (delta 5), pack-reused 53 (from 1)
Receiving objects: 100% (67/67), 2.24 MiB | 9.41 MiB/s, done.
Resolving deltas: 100% (19/19), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [2]:
mySeed = 350

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [3]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [4]:
import numpy as np
#La méthode simple pour estimer les coefficients θk dans un modèle linéaire est la régression linéaire par les moindres carrés ordinaires (MCO).
#Implémentation en Python avec NumPy :
# Charger les données d'entraînement
X = train_set['inputs']  # Matrice des prédicteurs
y = train_set['targets']  # Vecteur des cibles

# Ajouter une colonne de 1 à X pour le terme constant θ₀
X = np.c_[np.ones(X.shape[0]), X]

# Calculer les coefficients θk en utilisant la formule des MCO
theta_hat = np.linalg.solve(X.T @ X, X.T @ y)

# Afficher les coefficients estimés
print(f"Coefficients estimés : {theta_hat}")

Coefficients estimés : [17.43049692  3.44301622  3.59207667  7.17464915]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [6]:
import torch.nn as nn
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1) # 3 entrées (x, y, z) et 1 sortie (t)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [18]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()


# Optimisation avec Adam
optimizer = torch.optim.Adam(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

      # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = mySimpleNet(batch_inputs)
        # Reformater batch_targets
        batch_targets = batch_targets.view(-1, 1)  # ou .unsqueeze(1)
        loss = criterion(outputs, batch_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print progress (optional)
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [50/1000], Loss: 225.2488
Epoch [100/1000], Loss: 90.3679
Epoch [150/1000], Loss: 33.8583
Epoch [200/1000], Loss: 14.5070
Epoch [250/1000], Loss: 10.3194
Epoch [300/1000], Loss: 7.2576
Epoch [350/1000], Loss: 5.4750
Epoch [400/1000], Loss: 4.2758
Epoch [450/1000], Loss: 3.4151
Epoch [500/1000], Loss: 2.8871
Epoch [550/1000], Loss: 4.2548
Epoch [600/1000], Loss: 3.8551
Epoch [650/1000], Loss: 4.3257
Epoch [700/1000], Loss: 3.8962
Epoch [750/1000], Loss: 5.1113
Epoch [800/1000], Loss: 3.5526
Epoch [850/1000], Loss: 4.9035
Epoch [900/1000], Loss: 3.9349
Epoch [950/1000], Loss: 4.5287
Epoch [1000/1000], Loss: 4.1514


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [19]:
# Extraire les poids (θ₁, θ₂, θ₃)
weights = mySimpleNet.linear.weight.data
theta_1 = weights[0, 0].item()
theta_2 = weights[0, 1].item()
theta_3 = weights[0, 2].item()

# Extraire le biais (θ₀)
bias = mySimpleNet.linear.bias.data
theta_0 = bias[0].item()

# Afficher les estimations
print(f"θ₀ = {theta_0:.4f}")
print(f"θ₁ = {theta_1:.4f}")
print(f"θ₂ = {theta_2:.4f}")
print(f"θ₃ = {theta_3:.4f}")

θ₀ = 17.4299
θ₁ = 3.4450
θ₂ = 3.6006
θ₃ = 7.1686


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [20]:
# Charger les données de test
X_test = test_set['inputs']
y_test = test_set['targets']

# Ajouter une colonne de 1 à X_test pour le terme constant θ₀
X_test = np.c_[np.ones(X_test.shape[0]), X_test]

# Prédictions avec les estimations de la question 1 (MCO)
y_pred_mco = X_test @ theta_hat  # theta_hat est obtenu à la question 1

# Prédictions avec les estimations du réseau de neurones
X_test_tensor = torch.tensor(X_test[:, 1:], dtype=torch.float32)  # Convertir en tenseur PyTorch
y_pred_nn = mySimpleNet(X_test_tensor).detach().numpy().flatten()  # Prédictions du réseau

# Calculer l'erreur quadratique moyenne (MSE) pour les deux méthodes
mse_mco = np.mean((y_test - y_pred_mco)**2)
mse_nn = np.mean((y_test - y_pred_nn)**2)

# Afficher les résultats
print(f"MSE (MCO) : {mse_mco:.4f}")
print(f"MSE (Réseau de neurones) : {mse_nn:.4f}")

# Comparer les estimations des coefficients
print("\nCoefficients estimés (MCO) :", theta_hat)
print("Coefficients estimés (Réseau de neurones) :", [theta_0, theta_1, theta_2, theta_3])

MSE (MCO) : 4.0556
MSE (Réseau de neurones) : 4.0559

Coefficients estimés (MCO) : [17.43049692  3.44301622  3.59207667  7.17464915]
Coefficients estimés (Réseau de neurones) : [17.429948806762695, 3.444969654083252, 3.600615978240967, 7.168612003326416]


Le réseau de neurones, avec une architecture simple et un entraînement adéquat, a réussi à reproduire les résultats de la régression MCO pour cette relation linéaire, démontrant sa capacité d'apprentissage et de généralisation.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [21]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [22]:

# Q1: Type de réseau de neurones
# Il s'agit d'un réseau de neurones convolutif à une dimension (1D CNN)
# avec une architecture FCN (Fully Convolutional Network) et des convolutions causales.
# Il est conçu pour traiter des données temporelles.


# Calcul du nombre de paramètres de self.Down1 à la main:
# self.Down1 est une couche Down_causal avec les paramètres suivants :
#   - in_ch = 64
#   - out_ch = 128
# La couche Down_causal contient :
#  - une convolution 1D : 64 * 128 filtres de taille 4, plus un biais par filtre
#    soit 64*128*4 + 128 paramètres
#  - une couche de max pooling 1D avec un noyau de taille 2 et un pas de 2. (pas de paramètre)

# Donc, le nombre de paramètres dans self.Down1 est :
# (64 * 128 * 4) + 128 = 32768 + 128 = 32896

print("Nombre de paramètres dans self.Down1 :", 32896)


# Calcul du nombre total de paramètres du réseau :
model = causalFCN()
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Nombre total de paramètres dans le réseau :", total_params)

Nombre de paramètres dans self.Down1 : 32896
Nombre total de paramètres dans le réseau : 2872641


In [ ]:
# Nb de paramètres dans self.Down1: (calcul "à la main")

# Nb de paramètres au total:

**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

In [ ]:


# La taille du vecteur d'entrée est réduite par deux mécanismes principaux dans ce réseau :

# 1. Les convolutions 1D avec un noyau de taille > 1 et un padding < (kernel_size -1) : Chaque convolution réduit la taille de la séquence temporelle.
#    Dans ce réseau, le padding est toujours nul, et le kernel\_size est de taille 3 ou 5, cela entraine donc une reduction de la taille de la séquence.

# 2. Le max-pooling 1D : Cette opération réduit la taille du vecteur d'entrée en ne conservant que la valeur maximale dans une fenêtre glissante.
#    L'effet de réduction est déterminée par pooling_kernel_size et pooling_stride.

# La taille du vecteur est restituée dans la deuxième partie du réseau (partie "up") par :

# 1. Les opérations de Transposed Convolution (Up_causal): Ces opérations augmentent progressivement la taille du vecteur.
#    L'opération de upsampling est combinée à une convolution pour reconstruire l'information perdue pendant le downsampling.

# 2. Le concaténation avec les sorties correspondantes de la partie "downsampling" : Avant chaque opération de transposed convolution, l'activation courante est concaténée à la sortie de la couche correspondante dans la partie downsampling. Cela permet de fournir au décodeur des informations contextuelles à différentes échelles, ce qui améliore la reconstruction du vecteur et aide à recréer la taille d'origine.
#    L'opération permet de fournir au décodeur des informations contextuelles à différentes échelles.



**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?